## Log data munging and EDA

Just getting set up, testing

In [1]:
import pandas as pd
import numpy as np
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import sys
sys.path.insert(0,'../')

import mysticbit
from mysticbit import munging, ml, plots

Do you really want to drill me?
Do you really want to make me cry
AH AH AH AH


In [3]:
df_logs = munging.load_log_data()

df_ml = munging.create_ml_dataframe(df_logs, feature_lags = range(10), label_lags = range(5))

df_ml_train, df_ml_test = ml.train_test_split(df_ml)

Train well:  {'G10', 'G17', 'G12', 'B0700', 'G15', 'B08', 'G06', 'B12', 'G08', 'G070', 'B14', 'B200', 'G09', 'B13', 'G16'}
Test wells:  {'B06', 'B05', 'B03'}


In [4]:
X_cols = [c for c in df_ml_train.columns if 'lag' in c]
y_cols = [c for c in df_ml_train.columns if 'futr' in c]

print('X cols: ', X_cols)
print('y cols: ', y_cols)

X cols:  ['GR_lag_0', 'GR_lag_1', 'GR_lag_2', 'GR_lag_3', 'GR_lag_4', 'GR_lag_5', 'GR_lag_6', 'GR_lag_7', 'GR_lag_8', 'GR_lag_9']
y cols:  ['GR_futr_0', 'GR_futr_1', 'GR_futr_2', 'GR_futr_3', 'GR_futr_4']


In [5]:
models = ml.make_multiple_models(df_ml_train, X_cols, y_cols)

In [6]:
df_pred = ml.make_predictions(models, df_ml, X_cols, y_cols)

In [7]:
df_pred.head()

HACKANAME  TVDSS  RES_ID         GR   GR_lag_0   GR_lag_1   GR_lag_2  \
0       B03   1583  3005.0  58.601429  58.601429  49.958333  55.180000   
1       B03   1584  3005.0  51.303333  51.303333  58.601429  49.958333   
2       B03   1586  3005.0  52.490000  52.490000  51.303333  58.601429   
3       B03   1587  3005.0  53.040000  53.040000  52.490000  51.303333   
4       B03   1591  3005.0  58.120000  58.120000  53.040000  52.490000   

    GR_lag_3   GR_lag_4   GR_lag_5         ...          GR_futr_0_pred_base  \
0  57.498000  36.400000  34.288000         ...                    58.974140   
1  55.180000  57.498000  36.400000         ...                    51.412138   
2  49.958333  55.180000  57.498000         ...                    52.448244   
3  58.601429  49.958333  55.180000         ...                    53.634555   
4  51.303333  58.601429  49.958333         ...                    57.527183   

   GR_futr_1_pred_base  GR_futr_2_pred_base  GR_futr_3_pred_base  \
0            59.963371            59.428953            59.708259   
1            51.268785            52.598451            51.218921   
2            50.350406            49.896908            43.329452   
3            52.058860            48.701357            44.819100   
4            58.389747            55.913079            53.081308   

   GR_futr_4_pred_base  GR_futr_0_pred_low  GR_futr_1_pred_low  \
0            57.373458           58.336405           41.934717   
1            45.204185           51.380918           41.222971   
2            43.573999           52.256301           43.253630   
3            43.875806           52.748267           42.026078   
4            51.168514           58.016980           46.243175   

   GR_futr_2_pred_low  GR_futr_3_pred_low  GR_futr_4_pred_low  
0           43.679389           42.308312           41.411263  
1           39.340380           39.399134           36.534347  
2           41.957385           38.098932           37.544257  
3           37.617617           38.873492           36.255776  
4           44.714201           39.099303           38.818984  

[5 rows x 34 columns]

In [8]:
well_name = 'B05'
bit_depth = 1800

predictions = munging.get_log_predictions(df_pred, well_name, bit_depth)
predictions.head()

HACKANAME  TVDSS_bit_depth             pred_col      value  offset log_name  \
0       B05             1800  GR_futr_0_pred_high  54.753956     0.0       GR   
1       B05             1800  GR_futr_1_pred_high  66.869811     1.0       GR   
2       B05             1800  GR_futr_2_pred_high  76.086151     2.0       GR   
3       B05             1800  GR_futr_3_pred_high  79.911772     3.0       GR   
4       B05             1800  GR_futr_4_pred_high  81.684294     4.0       GR   

  model_name   TVDSS  
0       high  1800.0  
1       high  1801.0  
2       high  1802.0  
3       high  1803.0  
4       high  1804.0

In [9]:
df_logs = ml.create_facies(df_logs)

In [10]:
 
def plot_predictions(df_logs, df_pred, well_name, bit_depth):
    try:
        f, ax = plots.make_log_plot(df_logs, well_name, cols=['GR', 'RHOB', 'facies'])
        predictions = munging.get_log_predictions(df_pred, well_name, bit_depth)
        
        plots.add_predictions(ax[0], predictions)
        
        for axs in ax:
            axs.set_ylim(1800, 1760)
        plt.show()
        
        print('updated')
        
    except AssertionError:
        f, ax = plots.make_log_plot(df_logs, well_name)
        print('no predictions found at this depth in this well')


interact(plot_predictions, df_logs=fixed(df_logs), df_pred=fixed(df_pred),
         well_name=fixed(well_name), bit_depth=(1760, 1800, 1));

interactive(children=(IntSlider(value=1780, description='bit_depth', max=1800, min=1760), Output()), _dom_clas…

In [11]:
f, ax = plots.make_log_plot(df_logs, 'B05')
ax[0].scatter(predictions['value'], predictions['TVDSS'], marker='+')